# Regressione Logistica con Spark MLlib
In questo notebook vedremo come eseguire una semplice classificazione, utilizzando un modello di Regressione Logistica con il modulo MLlib di Spark. Il modello che andremo ha creare ha lo scopo di identificare tumori al seno maligni da delle informazioni estratte da delle agobiopsie.

## Procuriamoci il Dataset e importiamolo in un DataFrame
Possiamo scaricare il Dataset da [quesa pagina su Kaggle](https://www.kaggle.com/uciml/breast-cancer-wisconsin-data). Puoi anche scaricarlo senza uscire da Jupyter Notebook usando le  API di Kaggle per Python.

In [0]:
!kaggle datasets download uciml/breast-cancer-wisconsin-data

Il dataset è all'interno di un file ZIP, decomprimiamolo con unzip

In [0]:
!unzip breast-cancer-wisconsin-data.zip

Otterremo un file chiamato data.csv, che contiene il dataset, importiamolo in un DataFrame.

In [0]:
cancer_df = spark.read.csv("data.csv")
cancer_df.columns

Out[ 5 ]: 
['mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension',
 'malignant']

Il DataFrame ha 31 colonne:
* 30 features: che rappresentano delle proprietà dell'immagine, come raggio, area e perimetro.
* 1 target: che è la colonna malignant, un valore di 1 indica un tumore maligno, al contrario un valore di 0 indica un tumore benigno.

## Preprocessing dei dati
%md La classe MLlib richiede che le features si trovino tutte all'interno di un unico vettore su di una colonna, possiamo creare questa rappresentazione utilizzando la classe *VectorAssemlber* di MLlib:
* All'interno del parametro inputCols dobbiamo specificare quali sono le colonne con gli input.
* All'interno del parametro outputCol dobbiamo specificare il nome della colonna che conterrà le features.

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=cancer_df.columns[:-1], outputCol="features")
data_df = assembler.transform(cancer_df)

%md E' buona norma portare le features in un range di valori comuni, questo processo può velocizzare anche di molto la fase di addestramento. Facciamolo tramite la **standardizzazione** che ci permette di contenere le varie colonne all'interno di una distribuzione normale, cioè una distribuzione con media 0 e deviazione standard 1. Possiamo eseguire la standardizzazione usando la classe *StandardScaler* di MLlib.

In [0]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_df)
data_df = scaler_model.transform(data_df)

Ora possiamo creare i DataFrame per addestramento e test, estraendoli dal Dataframe originale, possiamo farlo tramite il metodo RandomSplit. Assegnamo il 70% degli esempi al set di addestramento e il 30% al set di test.

In [0]:
train_set, test_set = data_df.randomSplit([0.7, 0.3])

print("%d esempi nel train set" % train_set.count())
print("%d esempi nel test set" % test_set.count())

393 esempi nel train set
170 esempi nel test set

Ottimo ! Ora possiamo creare il modello di Regressione Logistica, usiamo la classe *LogisticRegression, all'interno del costruttore dovremo passare due parametri:
* **featuresCol**: il nome della colonna con le features
* **labelCol**: il nome della colonna con il target

In [0]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="scaled_features", labelCol="malignant")

Avviamo l'addestramento con il metodo *fit*, passando al suo interno il set di addetramento

In [0]:
model = lr.fit(train_set)

Abbiamo creato il nostro modello ! Ora verifichiamone la qualità.

## Valutiamo il Modello
Ora verifichiamone la qualità testandolo su dati che non ha visto durante l'addestramento, possiamo farlo usando il test set e il metodo *evalualte*.

In [0]:
evaluation = model.evaluate(test_set)

Il metodo *evaluate* calcolerà diverse metriche che ci possono aiutare a comprendere la qualità del modello, vediamone alcune.

#### Accuracy (Accuratezza)
L'accuracy indica semplicemente la percentuale di classificazioni che il nostro modello ha eseguito correttamente.

In [0]:
evaluation.accuracy

Out[ 10 ]: 0.956989247311828

### Precision (Precision)
La precision ci dice, tra le classificazioni eseguite per una data classe, quante sono effettivamente apparteneti a quella classe.

In [0]:
evaluation.precisionByLabel

Out[ 37 ]: [0.9545454545454546, 0.9519230769230769]

### Recall (Richiamo)
Il recall ci dice quanti dei casi positivi il modello è riuscito a classificare correttamente.

In [0]:
evaluation.recallByLabel

Out[ 40 ]: [0.9264705882352942, 0.9705882352941176]

## Testiamo il modello
Ora che abbiamo addestrato e validato il nostro modello, testiamolo su nuovi dati. Una clinica ci invia un file CSV contenente i risultati dell'agobiopsia per 6 pazienti che hanno in cura, dobbiamo utilizzare il nostro modello per identificare eventuali tumori maligni. Scarichiamo  il CSV

In [0]:
!wget https://raw.githubusercontent.com/ProfAI/bigdata/master/7%20-%20Machine%20Learning%20Supervisionato%20-%20Classificazione/data/exam_results.csv

Carichiamolo all'interno di un DataFrame.

In [0]:
exams_df = spark.read.csv("file:/databricks/driver/exam_results.csv", inferSchema=True, header=True)

Creiamo la colonna con le features

In [0]:
new_data_df = assembler.transform(exams_df)

Eseguiamo la standardizzazione

In [0]:
new_data_df = scaler_model.transform(new_data_df

E otteniamo le predizioni usando il metodo *transform* del modello

In [0]:
pred_df = model.transform(new_data_df)
pred_df.columns

Out[ 33 ]: 
['mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension',
 'features',
 'scaled_features',
 'rawPrediction',
 'probability',
 'prediction']

Come vedi il DataFrame risultante contiene due nuove colonne:
* **Prediction**: che contiene il label (0=benigno, 1=maligno)
* **Probabily**: che contiene la probabilità di apparteneza alle due classi.

In [0]:
pred_df.select(["probability", "prediction"]).show(6, False)

+------------------------------------------+----------+
probability |prediction|
+------------------------------------------+----------+
[1.0,0.0] |0.0 |
[1.7432636933686846E-244,1.0] |1.0 |
[7.453405721620013E-205,1.0] |1.0 |
[1.0,0.0] |0.0 |
[0.9999999999999991,8.075319188159663E-16]|0.0 |
[1.0,3.984631167625693E-159] |0.0 |
+------------------------------------------+----------+

Due dei tumori sono stati etichettati come maligni, con delle probabilità associate molto alte, sarebbe il caso che la clinica effettuasse ulteriori esami.